# Movie Recommendation System

## Library Imports and Constants Initialization

In [1]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors

## Data Loading

In [2]:
movies_df = pd.read_csv(os.path.join(os.getcwd(), 'data', "movies.csv"))
ratings_df = pd.read_csv(os.path.join(os.getcwd(), 'data', "ratings.csv")).iloc[:500000, :] # Using 500,000 for now due to sheer size of original
tags_df = pd.read_csv(os.path.join(os.getcwd(), 'data', "tags.csv"))
genome_scores_df = pd.read_csv(os.path.join(os.getcwd(), 'data', "genome-scores.csv"))
genome_tags_df = pd.read_csv(os.path.join(os.getcwd(), 'data', "genome-tags.csv"))

## Dataset Exploration

### Movies Info

In [3]:
print("\nGeneral information of the movies' dataset:")
print(movies_df.info())

print("------------------------------------------------------------------")

print("\nHead of the movies' dataset:")
print(movies_df.head())

print("------------------------------------------------------------------")

print("\nShape of the movies' dataset:", movies_df.shape)

print("------------------------------------------------------------------")

print("\nMissing values of the movies' dataset:")
print(movies_df.isnull().sum())


General information of the movies' dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB
None
------------------------------------------------------------------

Head of the movies' dataset:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3            

### Ratings Info

In [83]:
print("\nGeneral information of the ratings' dataset:")
print(ratings_df.info())
# Timestamp unnecessary

print("------------------------------------------------------------------")

print("\nHead of the ratings' dataset:")
print(ratings_df.head())

print("------------------------------------------------------------------")

print("\nShape of the ratings' dataset:", ratings_df.shape)

print("------------------------------------------------------------------")

print("\nMissing values of the ratings' dataset:")
print(ratings_df.isnull().sum())


General information of the ratings' dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     500000 non-null  int64  
 1   movieId    500000 non-null  int64  
 2   rating     500000 non-null  float64
 3   timestamp  500000 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 15.3 MB
None
------------------------------------------------------------------

Head of the ratings' dataset:
   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510
------------------------------------------------------------------

Shape of the ratings' dataset: (500000, 4)
------------------------------------------------------------------

Missing values of the ratings' 

### Tags Info

In [84]:
print("\nGeneral information of the tags' dataset:")
print(tags_df.info())
# Timestamp unnecessary
# Tags are user defined, so a lot of them (although the same) will be spelled differently. 
    # i.e. sci-fi vs scifi, 90's vs 90s, and Horror vs horror

print("------------------------------------------------------------------")

print("\nHead of the tags' dataset:")
print(tags_df.head())

print("------------------------------------------------------------------")

print("\nShape of the tags' dataset:", tags_df.shape)
print("------------------------------------------------------------------")

print("\nMissing values of the tags' dataset:")
print(tags_df.isnull().sum())
# 16 missing values

print("------------------------------------------------------------------")

print("\nUnique values tags' in dataset:")
print(tags_df.tag.nunique())


General information of the tags' dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093360 entries, 0 to 1093359
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1093360 non-null  int64 
 1   movieId    1093360 non-null  int64 
 2   tag        1093344 non-null  object
 3   timestamp  1093360 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 33.4+ MB
None
------------------------------------------------------------------

Head of the tags' dataset:
   userId  movieId               tag   timestamp
0       3      260           classic  1439472355
1       3      260            sci-fi  1439472256
2       4     1732       dark comedy  1573943598
3       4     1732    great dialogue  1573943604
4       4     7569  so bad it's good  1573943455
------------------------------------------------------------------

Shape of the tags' dataset: (1093360, 4)
---------------------------------------------

## Initial Data Preprocessing for Ratings, and Tags

### Ratings Preprocessing

In [85]:
print(ratings_df.head())
ratings_cleaned = ratings_df.copy()
ratings_cleaned = ratings_cleaned.drop(columns=['timestamp'])
ratings_cleaned = ratings_cleaned[ratings_cleaned['rating'] >= 4.0]
print(ratings_cleaned.head())


   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510
   userId  movieId  rating
0       1      296     5.0
2       1      307     5.0
3       1      665     5.0
5       1     1088     4.0
8       1     1237     5.0


### Tags Preprocessing

In [86]:
def preprocess_tags(tag):
    """Preprocess tags by lowercasing, removing parentheses, hyphens, and 'based'."""
    if pd.isna(tag):
        return tag  # Return NaN as is
    tag = tag.lower()
    tag = re.sub(r'\(.*?\)', '', tag)
    tag = tag.replace('-', '')
    return tag.strip()

In [87]:
tags_cleaned = tags_df.copy()
print(tags_df.head())

tags_cleaned = tags_cleaned.drop(columns=['timestamp'])
tags_cleaned = tags_cleaned.dropna()

tags_cleaned['tag'] = tags_cleaned['tag'].apply(preprocess_tags)
tags_cleaned = tags_cleaned[~tags_cleaned['tag'].str.contains('based', na=False)]
print(tags_cleaned.head())

   userId  movieId               tag   timestamp
0       3      260           classic  1439472355
1       3      260            sci-fi  1439472256
2       4     1732       dark comedy  1573943598
3       4     1732    great dialogue  1573943604
4       4     7569  so bad it's good  1573943455
   userId  movieId               tag
0       3      260           classic
1       3      260             scifi
2       4     1732       dark comedy
3       4     1732    great dialogue
4       4     7569  so bad it's good


### Movies Preprocessing

In [88]:
def preprocess_movies(genre):
    """Convert genre strings into lowercase."""
    return genre.lower()  # Make sure to return the processed genre

In [89]:
movies_cleaned = movies_df.copy()  # Create a copy to avoid modifying the original DataFrame
print(movies_df.head())

movies_cleaned['genres'] = movies_cleaned['genres'].apply(preprocess_movies)
print(movies_cleaned.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  adventure|animation|children|comedy|fantasy  
1     

## Merging datasets to create Movies-based dataset, movies_genres&tags

### Creating initial movies_genres&tags dataset

In [90]:
# Merge movies and tags data
merged_data = pd.merge(movies_cleaned, tags_cleaned, on='movieId', how='left')
def combine_and_deduplicate(group):
    # Combine all genres and tags for each group (movie)
    all_genres = group['genres'].iloc[0].split('|') if pd.notna(group['genres'].iloc[0]) else []
    all_tags = group['tag'].dropna().tolist()
    combined = list(set(all_genres + all_tags))  # Remove duplicates
    return '|'.join(combined)

# Apply the function to each group
movies_genres_tags = merged_data.groupby('movieId').apply(combine_and_deduplicate).reset_index(name='genres&tags')

# Merge with movies_cleaned to get titles
movies_genres_tags = pd.merge(movies_cleaned[['movieId', 'title']], movies_genres_tags, on='movieId')

print(movies_genres_tags.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                         genres&tags  
0  daring rescues|cartoon|imdb top 250|clever|ame...  
1  scary|kids|time travel|clv|jungle|see also:zat...  
2  sequel|old|sequel fever|daryl hannah|clv|sophi...  
3  comedy|romance|single mother|revenge|divorce|c...  
4  sequel|sequel fever|diane keaton|clv|daughter|...  


### Adding relevance scores for each genre/tag to the movies-based dataset

In [92]:
# Merge genome scores and tags
genome_combined = pd.merge(genome_scores_df, genome_tags_df, on='tagId')

# Creating a dictionary for quick relevance score lookup
genome_dict = genome_combined.set_index(['movieId', 'tag'])['relevance'].to_dict()

In [93]:
# Function to find relevance scores using the dictionary
def find_relevance_scores(row):
    genres_tags = row['genres&tags'].split('|')
    scores = [genome_dict.get((row['movieId'], tag), 0) for tag in genres_tags]
    # Filter out scores below 0.6 and corresponding tags/genres
    filtered_data = [(tag, score) for tag, score in zip(genres_tags, scores) if score >= 0.7]
    if filtered_data:
        filtered_tags, filtered_scores = zip(*filtered_data)
        return '|'.join(filtered_tags), ','.join(map(str, filtered_scores))
    else:
        return '', ''

# Apply the function to movies_genres_tags
movies_genres_tags[['genres&tags', 'relevance_scores']] = movies_genres_tags.apply(
    find_relevance_scores, axis=1, result_type='expand')

# Save the updated dataset in the data folder
movies_genres_tags.to_csv('data/movies_genres&tags.csv', index=False)

## Merging datasets to create users-based dataset, users_liked_genres&tags

In [91]:
# Merge ratings_cleaned with movies_cleaned
user_movies_genres = pd.merge(ratings_cleaned, movies_cleaned, on='movieId')

# Merge the above with tags_cleaned
user_movies_genres_tags = pd.merge(user_movies_genres, tags_cleaned, on=['userId', 'movieId'], how='left')

# Function to concatenate genres and user-specific tags
def concatenate_user_genres_tags(group):
    genres = '|'.join(group['genres'])
    tags = '|'.join(group['tag'].dropna().unique())  # Drop NA and get unique tags
    return genres + ('|' + tags if tags else '')  # Combine genres and tags, separated by '|'

# Aggregate genres and tags for each user
users_liked_genres_tags = user_movies_genres_tags.groupby('userId').apply(concatenate_user_genres_tags).reset_index(name='liked_genres&tags')

users_liked_genres_tags.to_csv('data/users_liked_genres&tags.csv', index=False)

### Returning Recommendations to User

In [4]:
# Sample 2D array [[movie_names], [tags_embeddings]]
sample_2D_array = [['Movie A', 'Movie B', 'Movie C', 'Movie D', 'Movie E', 'Toy Story', 'Hunchback', 'John Wick'],
    [
        np.mean([[0.1, 0.2, 0.3], [0.4, 0.3, 0.5]], axis=0),
        np.mean([[0.3, 0.6, 0.1], [0.2, 0.1, 0.4]], axis=0),
        np.mean([[0.5, 0.8, 0.5], [0.1, 0.3, 0.3]], axis=0),
        np.mean([[0.2, 0.2, 0.2], [0.3, 0.5, 0.6]], axis=0),
        np.mean([[0.7, 0.8, 0.9], [0.2, 0.1, 0.2]], axis=0),
        np.mean([[0.1, 0.2, 0.3], [0.4, 0.3, 0.5]], axis=0),
        np.mean([[0.1, 0.2, 0.3], [0.6, 0.2, 0.8]], axis=0),
        np.mean([[0.9, 0.1, 0.4], [0.3, 0.8, 0.7]], axis=0)
    ]]

# User's liked movies 2D array
user_2D_array = [['Toy Story', 'Hunchback', 'John Wick'],
    [
        np.mean([[0.1, 0.2, 0.3], [0.4, 0.3, 0.5]], axis=0),
        np.mean([[0.1, 0.2, 0.3], [0.6, 0.2, 0.8]], axis=0),
        np.mean([[0.9, 0.1, 0.4], [0.3, 0.8, 0.7]], axis=0)
    ]]

# Creating constant names for better clarity
sample_movies = sample_2D_array[0]
sample_tag_embeddings = sample_2D_array[1]

user_movies = user_2D_array[0]
user_tag_embeddings = user_2D_array[1]

# Convert tag embeddings to a 2D NumPy array
X_sample = np.array(sample_tag_embeddings)
X_user = np.array(user_tag_embeddings)

# Create and fit the model with cosine similarity using 10 closest movies
# (4 in this example as there are not 10 other movies)
knn = NearestNeighbors(n_neighbors=4, metric='cosine')
knn.fit(X_sample)

# Find 4 movies similar to the query movie
recommendations = []
for test in X_user:
    distances, indices = knn.kneighbors([test])

    # Filter out movies that are already liked by the user
    similar_movies = [sample_movies[i] for i in indices[0] if sample_movies[i] not in user_movies]
    recommendations.append(similar_movies)

# Print the similar movies
for i, user_movie in enumerate(user_movies):
    print(f"Based on {user_movie}, we would recommend:")
    for recommended_movie in recommendations[i]:
        print(recommended_movie)
    print()


NameError: name 'NearestNeighbors' is not defined